In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv("../input/suspects.csv")

Exploratory Analysis
--------------------

In [ ]:
data[0:5]

In [ ]:
# Citizenship Status
data['citizenship_status'].value_counts(normalize=True)

Does illegal immigrant the source of terrorism? No! Illegal Immigrant only counts for 2% of all terrorists. On the contrary, US Born Citizen accounts for 47% of terrorists, followed by Naturalized Citizen (20%), and Permanent Resident (10%), which sum up to 77%: 77% of terrorism activities is linked with Long-term, legal residents. 

In [ ]:
# Gender
data['sex'].value_counts(normalize=True)

Terrorism activities are predominantly conducted by males.

In [ ]:
# Marital Status
data['marital_status'].value_counts(normalize=True)

In [ ]:
#Last Resident Country and state
print(data['last_residency_state'].value_counts())
print(data['last_residency_country'].value_counts())

In [ ]:
Data Transformation
-------------------

In [ ]:
#Transfer factor variables into dummy variables
crit = ['informant_use','public_tip','family/community_tip','israel/judaism_target','military_target',\
       'awlaki_contact','foreign_militant_contact','online_redicalization','overseas_military_training',\
       'us_military_experience','sex','marital_status']
for column in data:
    if column in crit:
        data = data.merge(pd.get_dummies(data=data[column],prefix=column),right_index=True,left_index=True)
        del data[column]
        

In [ ]:
#Fill nas
data['age'] = data['age'].fillna(data['age'].mean())

In [ ]:
#Select columns of use
useful_columns = ['age',
       'citizenship_status', 
       'informant_use_No', 'informant_use_Yes', 'public_tip_No',
       'public_tip_Yes', 'family/community_tip_No', 'family/community_tip_Yes',
       'israel/judaism_target_No', 'israel/judaism_target_Yes',
       'military_target_No', 'military_target_Yes', 'awlaki_contact_Contact',
       'awlaki_contact_None', 'awlaki_contact_Ties',
       'foreign_militant_contact_No', 'foreign_militant_contact_Yes',
       'overseas_military_training_No', 'overseas_military_training_Yes',
       'us_military_experience_No', 'us_military_experience_Yes', 'sex_Female',
       'sex_Male', 'marital_status_Divorced', 'marital_status_Married',
       'marital_status_Split', 'marital_status_Unknown',
       'marital_status_Unmarried', 'marital_status_Widowed']
data_for_model = data[useful_columns]


In [ ]:
#Set independent variables and dependent variables
x = data_for_model.drop('citizenship_status',axis=1)
y = data_for_model['citizenship_status'].tolist()
le = preprocessing.LabelEncoder()
le.fit(y)
list(le.classes_)
y_transformed = le.transform(y) 

In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x,y)

In [ ]:
#Feature weight
index = list(le.classes_)
column_name = list(x.keys())
result = pd.DataFrame(lr.coef_,index=index,columns=column_name)
result.to_csv('result.csv')

In [ ]:
result

In [ ]:
#Calculate acc on training set
acc = lr.score(x, y)
print("Accuracy: {:.4f}".format(acc))